# Converting ADO to Cabrillo 

RUMLOGng is an excellent logger - I recently used it for some causual contesting in CQWW-2023 CW. I wanted to 
work some new countries - and by having my QSO history, made this quite easy.                                         
However, I soon worked over 40 stations, so I needed to do 2 things

  - Generate a Cabrillo file
  - Update my QSO's via LOTW.

Well the LOTW update is simple - the logger will do that. But to generate the Cabrillo file, needed a little work.

## Extract the QSO's 

Just select the QSO's you want

![Select](./select.png)

After this choose 

![Export](./export.png)

And save the ADO data. In this case I am using the filename **WW-2023**, RUMLogng will append the file type of *adx* to the file.

## What is ADO data ??

It essentially is XML, and if that means nothing to you - just think of it as *structured* text.

## Why not ADIF ?
Adif it seems does not export the frequency by default, it exports the Band. I did look at adif first, but this is why the ADO/ADX option was used.


## Python/Programming 

**Danger** code ahead. 

I hope this is easy to read, please feel free to update/use as you wish.

In [12]:
from bs4 import BeautifulSoup # A standard Structured Text Parser in Python
# Reading the data inside the xml
# file to a variable under the name of data
with open('WW-2023.adx', 'r') as f:
    data = f.read()

In [2]:
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object 
Bs_data = BeautifulSoup(data, "xml")

# Tag  
We can iterate through the data using find_all, we want to see 'RECORD' .... 

    for tag in Bs_data.find_all('RECORD'):
       .. do something

inside a 'tag' we can access a field by

    tag.<FIELD> 

However this looks like this 

    <CALL>4W8X</CALL> 

To get just the call 

    tag.<FIELD>.text


So to get the *Call* we do  

    tag.<CALL>.text

and we extract

    '4W8X'

In [13]:
# Lets see all the Callsigns in our ADX file, which has been parsed by BeautifulSoup
for tag in Bs_data.find_all('RECORD'):
    print(f"Rec {tag.CALL.text}")
    

Rec 3W9A
Rec VK9XY
Rec IR2Q
Rec D4C
Rec 4D3X
Rec FW5N
Rec E2X
Rec LN8W
Rec CR3A
Rec S57K
Rec P3AA
Rec VR2T
Rec HI3Y
Rec P44W
Rec C6AQQ
Rec K6LL
Rec XR1EW
Rec CO8ZZ
Rec PX2A
Rec FY5KE
Rec V31CQ
Rec V26K
Rec YR0K
Rec 3B8M
Rec VU2NXG
Rec 9A1P
Rec V85NPV
Rec DF0HQ
Rec HB9CA
Rec CR3A
Rec S58M
Rec LZ9W
Rec ER1KAA
Rec 3W9A
Rec EW5A
Rec OM8CW
Rec YT8A
Rec OL9Z
Rec S77HQ
Rec 3B7M
Rec DX8H
Rec 5W1SA
Rec 3B9KW
Rec D4C
Rec 3B8M
Rec 3B8M
Rec MU6P
Rec EY7BJ
Rec KP2M
Rec LU8DPM
Rec NP2J
Rec CE3CT
Rec 4D3X
Rec KP2B
Rec 3G6EW
Rec OA4EA
Rec CW5W
Rec XE2AD
Rec AA7A
Rec XE1AY
Rec PJ5J
Rec VE7SZ
Rec K6LL
Rec HK3TU
Rec K0FX
Rec HQ9X
Rec TI7W
Rec V47T
Rec K8R
Rec W7WA
Rec HQ9X
Rec 4D3X
Rec HG6N
Rec OM2XW
Rec TM7A
Rec YT7A
Rec UA0DAR
Rec HL2BQG
Rec AH2R
Rec LX7I
Rec OH0X
Rec IM0HRP
Rec XW4DX
Rec EI6FR
Rec DU3T
Rec DU3T
Rec LX7I
Rec EX0M
Rec 9M8YY
Rec ER2QA
Rec V85NPV
Rec K8R
Rec AH2R
Rec ER1KAA
Rec NP4Z
Rec ZF1A
Rec VK9XY
Rec FY5KE
Rec PJ2T
Rec ZF5T
Rec KH0W
Rec WH2JA
Rec P44W
Rec PJ4K
Rec TI7W
Rec KP2M
Rec H

# Fields 

We need the following fields. Please note that as CQWW using the *CQ Zone*, we have taken what the logger
generated for us (on the assumption that the call was resolved correctly). If you were recording the Exchange 
say into the *notes* field. Then you would have to extract from that field and not the CQZ.

 - QSO_DATE
 - TIME_ON 
 - FREQ 
 - MODE 
 - CALL 
 - CQZ 


In [4]:
fields=['QSO_DATE','TIME_ON','FREQ','MODE','CALL','CQZ']

In [5]:
tag.find(fields[0]).text

'20231115'

In [15]:
# These are my internal definitions of Data for Cabrillo
# It makes data munching a little easier.
from hamcabrillo import cabrecord
from hamcabrillo import LoadCab

# You will need to edit this to your own needs.
preamble='''
START-OF-LOG: 3.0
CALLSIGN: DV3A
CONTEST: CQ-WW-CW
CATEGORY-OPERATOR: SINGLE-OP
CATEGORY-ASSISTED: ASSISTED
CATEGORY-BAND: ALL
CATEGORY-POWER: HIGH
CATEGORY-MODE: CW
CATEGORY-TRANSMITTER: ONE
CERTIFICATE: YES
CLAIMED-SCORE: 0
CLUB:
LOCATION: DX
CREATED-BY: TimLog
NAME: Tim Seed
ADDRESS: 017 Arayat Magalang Rd
ADDRESS: Purok 1
ADDRESS: Brgy Arenas Arayat
ADDRESS: 2012 Pampanga
ADDRESS: Philippines 
OPERATORS: DU3TW
SOAPBOX: Just S&P. Some JA and BY stations have VERY wide and noisy signals.
SOAPBOX: Which is masking the DX stations, by their splatter.'''



example='''QSO:  7007 CW 2016-11-26 0212 A45WG         599 21     LZ3ZZ         599 20     0
QSO:  7012 CW 2016-11-26 0215 A45WG         599 21     IR2L          599 15     0
QSO:  7013 CW 2016-11-26 0216 A45WG         599 21     SM5F          599 14     0
QSO:  7016 CW 2016-11-26 0217 A45WG         599 21     UN9L          599 17     0'''

postamble='END-OF-LOG:'


In [16]:
n=1
recs=[]
for tag in Bs_data.find_all('RECORD'):
    rec=cabrecord(float(tag.FREQ.text),tag.MODE.text,tag.QSO_DATE.text+tag.TIME_ON.text,'DV3A','599','27',tag.CALL.text,'599',tag.CQZ.text,n)
    recs.append(rec)

with open('WW-2023.cbr','wt') as ofp:
    ofp.write(f"{preamble}\n")
    #Used for Dev/Testing
    #ofp.write(f"{example}\n")
    # For some strange reason - the export appears to be in reverse order.
    # So we reverse the array
    for r in reversed(recs):
        ofp.write(f"QSO: {r.freq*1000:5.0f} {r.mode:2s} {r.when[0:4]:4s}-{r.when[4:6]:2s}-{r.when[6:8]:2s} {r.when[8:12]:4s} {r.mycall:13s} {r.myrst:3s} {r.myexch:6s} {r.dxcall:13s} {r.dxrst:3s} {r.dxexch:6s} 0\n")
    ofp.write(f"{postamble}")
print("Please look at the file 'WW-2023.cbr' for the converted data")

Please look at the file 'WW-2023.cbr' for the converted data


## Conclusion

RHUMLogng is a great logger, with a decent data interface. 
Manipulating its output is quite easy.


Using the excellent site [http://tools.adventureradio.de/analyzer/](http://tools.adventureradio.de/analyzer/)

This was my DX map 

![Dx.png](./Dx.png)